In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!ls "/content/drive/My Drive/data_set/"

census.csv  house-prices.csv


In [3]:
import pandas as pd
base = pd.read_csv('drive/My Drive/data_set/census.csv')
base.head()

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
base['income'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [0]:
def converte_classe(rotulo):
  if rotulo == ' >50K':
    return 1
  else:
    return 0

In [0]:
base['income'] = base['income'].apply(converte_classe)# convertendo as classes de classificacao em '0' ou '1'

In [7]:
base.head()

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [0]:
x = base.drop('income', axis = 1) # 'axis = 1' apaga toda a coluna, deixa só os atributos previsores
y = base['income']

In [9]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: income, dtype: int64

In [0]:
from sklearn.model_selection import train_test_split
x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x, y, test_size=0.3)

In [11]:
x_treinamento.shape[1]

14

In [12]:
y_treinamento.shape

(22792,)

In [13]:
x_teste.shape

(9769, 14)

In [14]:
base.columns

Index(['age', 'workclass', 'final-weight', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loos', 'hour-per-week', 'native-country',
       'income'],
      dtype='object')

In [0]:
import tensorflow as tf

In [0]:
workclass = tf.feature_column.categorical_column_with_hash_bucket(key = 'workclass', hash_bucket_size=100)
education = tf.feature_column.categorical_column_with_hash_bucket(key = 'education', hash_bucket_size=100)
marital_status = tf.feature_column.categorical_column_with_hash_bucket(key = 'marital-status', hash_bucket_size=100)
occupation = tf.feature_column.categorical_column_with_hash_bucket(key = 'occupation', hash_bucket_size=100)
relationship = tf.feature_column.categorical_column_with_hash_bucket(key = 'relationship', hash_bucket_size=100)
race = tf.feature_column.categorical_column_with_hash_bucket(key = 'race', hash_bucket_size=100)
country = tf.feature_column.categorical_column_with_hash_bucket(key = 'native-country', hash_bucket_size=100)

In [17]:
base.sex.unique()

array([' Male', ' Female'], dtype=object)

In [0]:
sex = tf.feature_column.categorical_column_with_vocabulary_list(key='sex',vocabulary_list=[' Male', ' Female'])

In [19]:
base.columns

Index(['age', 'workclass', 'final-weight', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loos', 'hour-per-week', 'native-country',
       'income'],
      dtype='object')

In [20]:
base.age.mean()

38.58164675532078

In [21]:
base.age.std()

13.640432553581146

In [0]:
def padroniza_age(valor):
  return tf.divide(tf.subtract(tf.cast(valor, tf.float32),tf.constant(38.58)),tf.constant(13.63))

In [0]:
age = tf.feature_column.numeric_column(key='age',normalizer_fn=padroniza_age )
final_weight = tf.feature_column.numeric_column(key='final-weight')
education_num = tf.feature_column.numeric_column(key='education-num')
capital_gain = tf.feature_column.numeric_column(key='capital-gain')
capital_loos = tf.feature_column.numeric_column(key='capital-loos')
hour = tf.feature_column.numeric_column(key='hour-per-week')

In [0]:
colunas = [age, workclass, final_weight, education, education_num,
          marital_status, occupation, relationship, race, sex,
          capital_gain, capital_loos, hour, country]
      

In [0]:
embedded_workclass =      tf.feature_column.embedding_column(workclass, dimension=len(base.workclass.unique()))
embedded_education =      tf.feature_column.embedding_column(education, dimension=len(base.education.unique()))
embedded_marital_status = tf.feature_column.embedding_column(marital_status, dimension=len(base['marital-status'].unique()))
embedded_occupation =     tf.feature_column.embedding_column(occupation, dimension=len(base.occupation.unique()))
embedded_relationship =   tf.feature_column.embedding_column(relationship, dimension=len(base.relationship.unique()))
embedded_race =           tf.feature_column.embedding_column(race, dimension=len(base.race.unique()))
embedded_sex =            tf.feature_column.embedding_column(sex, dimension=len(base.sex.unique()))
embedded_country =        tf.feature_column.embedding_column(country, dimension=len(base['native-country'].unique()))

In [0]:
colunas_rna = [age, embedded_workclass, final_weight, embedded_education, education_num,
          embedded_marital_status, embedded_occupation, embedded_relationship, embedded_race, embedded_sex,
          capital_gain, capital_loos, hour, embedded_country]

In [27]:
funcao_treinamento = tf.estimator.inputs.pandas_input_fn(x = x_treinamento, y = y_treinamento, batch_size=32, num_epochs=None, shuffle=True)
classificador = tf.estimator.DNNClassifier(hidden_units=[8,8], feature_columns=colunas_rna,n_classes=2)
classificador.train(input_fn=funcao_treinamento, steps=10000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpffnr3hn9', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4c60e4c5c0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, use the `tf.

In [0]:
funcao_teste = tf.estimator.inputs.pandas_input_fn(x = x_teste, y = y_teste, batch_size=32, num_epochs=1, shuffle=False)

In [29]:
classificador.evaluate(input_fn=funcao_teste)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-06T23:55:18Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /tmp/tmpffnr3hn9/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-06-06-23:55:20
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.7671205, accuracy_baseline = 0.762207, auc = 0.5105452, auc_precision_recall = 0.6169386, average_loss = 0.6021386, global_step = 10000, label/mean = 0.23779301, loss = 19.223177, precision = 0.98, prediction/mean = 0.24207054, recall = 0.021093413
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: /tmp/tmpffnr3hn9/model.ckpt-10000


{'accuracy': 0.7671205,
 'accuracy_baseline': 0.762207,
 'auc': 0.5105452,
 'auc_precision_recall': 0.6169386,
 'average_loss': 0.6021386,
 'global_step': 10000,
 'label/mean': 0.23779301,
 'loss': 19.223177,
 'precision': 0.98,
 'prediction/mean': 0.24207054,
 'recall': 0.021093413}